In [1]:
import sqlalchemy
import pandas as pd

In [2]:
from pathlib import Path
import requests

In [3]:
db_data = requests.get('https://github.com/messa/movie-db-2020/raw/master/movies.sqlite').content
Path('movies.sqlite').write_bytes(db_data)

1421312

In [54]:
engine = sqlalchemy.create_engine('sqlite:///movies.sqlite') #sqlite is protocol, like https://

In [5]:
sqlalchemy.inspect(engine).get_table_names()

['actors', 'movie_to_actor', 'movies']

In [6]:
def prozkoumat(engine):
    inspector = sqlalchemy.inspect(engine)
    for schema in inspector.get_schema_names():
        print('Schema:', schema)
        for table_name in inspector.get_table_names(schema=schema):
            print()
            print('  Table:', table_name)
            print()
            for column in inspector.get_columns(table_name, schema=schema):
                print('      Column:', column['name'].ljust(12), column['type'])

In [7]:
prozkoumat(engine)

Schema: main

  Table: actors

      Column: id           INTEGER
      Column: csfd_url     VARCHAR
      Column: name         VARCHAR
      Column: birth_date   DATE

  Table: movie_to_actor

      Column: id           INTEGER
      Column: movie_id     INTEGER
      Column: actor_id     INTEGER

  Table: movies

      Column: id           INTEGER
      Column: title        VARCHAR
      Column: csfd_url     VARCHAR
      Column: year         INTEGER
      Column: rating       NUMERIC


0. Najdi nejlepší devadesátkové filmy - t.j. filmy, které vyšly v letech 1990-1999 a mají hodnocení minimálně 85. Seřaď sestupně od nejlepšího. (Tip: zkus si dohledat použití operátoru BETWEEN).

In [10]:
list(engine.execute('SELECT * FROM movies WHERE year BETWEEN 1900 AND 1999 AND rating >= 85 ORDER BY rating DESC'))

[(1, 'Vykoupení z věznice Shawshank', 'https://www.csfd.cz/film/2294-vykoupeni-z-veznice-shawshank/', 1994, 95.3),
 (2, 'Forrest Gump', 'https://www.csfd.cz/film/10135-forrest-gump/', 1994, 94.5),
 (3, 'Zelená míle', 'https://www.csfd.cz/film/2292-zelena-mile/', 1999, 92.8),
 (4, 'Přelet nad kukaččím hnízdem', 'https://www.csfd.cz/film/2982-prelet-nad-kukaccim-hnizdem/', 1975, 92.5),
 (5, 'Sedm', 'https://www.csfd.cz/film/2671-sedm/', 1995, 92.4),
 (6, 'Schindlerův seznam', 'https://www.csfd.cz/film/8653-schindleruv-seznam/', 1993, 92.3),
 (7, 'Kmotr', 'https://www.csfd.cz/film/1644-kmotr/', 1972, 91.8),
 (9, 'Dvanáct rozhněvaných mužů', 'https://www.csfd.cz/film/6178-dvanact-rozhnevanych-muzu/', 1957, 91.3),
 (10, 'Pelíšky', 'https://www.csfd.cz/film/4570-pelisky/', 1999, 91.2),
 (11, 'Kmotr II', 'https://www.csfd.cz/film/1645-kmotr-ii/', 1974, 90.8),
 (12, 'Terminátor 2: Den zúčtování', 'https://www.csfd.cz/film/1248-terminator-2-den-zuctovani/', 1991, 90.8),
 (13, 'Pulp Fiction: His

1. Najdi film, ve kterém účinkuje nejvíce herců.

In [19]:
list(engine.execute('''
        SELECT title, count(*) 
        FROM     
            (SELECT ma.*, a.name, m.title
            FROM movie_to_actor AS ma
            LEFT JOIN actors AS a ON a.id = ma.actor_id
            LEFT JOIN movies AS m ON m.id = ma.movie_id)
        GROUP BY title
        ORDER BY count(*) DESC
        LIMIT 1
        '''))

[('Adéla ještě nevečeřela', 154)]

2. Najdi herce, který hraje v nejvíce filmech.

In [20]:
list(engine.execute('''
        SELECT name, count(*) 
        FROM     
            (SELECT ma.*, a.name, m.title
            FROM movie_to_actor AS ma
            LEFT JOIN actors AS a ON a.id = ma.actor_id
            LEFT JOIN movies AS m ON m.id = ma.movie_id)
        GROUP BY name
        ORDER BY count(*) DESC
        LIMIT 1
        '''))

[('Rudolf Hrušínský', 15)]

3. Zkus zjistit, zda existuje v databázi film, ve kterém nehraje žádný herec.

In [21]:
list(engine.execute('''
        SELECT title, count(*) 
        FROM     
            (SELECT ma.*, a.name, m.title
            FROM movie_to_actor AS ma
            LEFT JOIN actors AS a ON a.id = ma.actor_id
            LEFT JOIN movies AS m ON m.id = ma.movie_id)
        WHERE name is NULL
        GROUP BY title
        ORDER BY count(*) DESC       
        '''))

[]

In [35]:
list(engine.execute('''
        SELECT *
        FROM        
            (SELECT m.title, ma.*, a.name 
            FROM movies AS m
            LEFT JOIN movie_to_actor AS ma ON ma.movie_id = m.id
            LEFT JOIN actors AS a ON a.id = ma.actor_id)
        WHERE name IS NULL
        '''))

[]

In [33]:
list(engine.execute('''
        SELECT *
        FROM        
            (SELECT m.title, ma.*, a.name, a.birth_date 
            FROM movies AS m
            LEFT JOIN movie_to_actor AS ma ON ma.movie_id = m.id
            LEFT JOIN actors AS a ON a.id = ma.actor_id)
        WHERE birth_date IS NULL
        '''))

[('Vykoupení z věznice Shawshank', 18, 1, 18, 'Renee Blaine', None),
 ('Vykoupení z věznice Shawshank', 21, 1, 21, 'Brian Libby', None),
 ('Vykoupení z věznice Shawshank', 22, 1, 22, 'Dion Anderson', None),
 ('Vykoupení z věznice Shawshank', 25, 1, 25, 'Morgan Lund', None),
 ('Vykoupení z věznice Shawshank', 26, 1, 26, 'Brian Brophy', None),
 ('Vykoupení z věznice Shawshank', 27, 1, 27, 'V.J. Foster', None),
 ('Forrest Gump', 39, 2, 39, 'Brett Rice', None),
 ('Forrest Gump', 40, 2, 40, 'Kirk Ward', None),
 ('Forrest Gump', 44, 2, 44, 'Michael McFall', None),
 ('Forrest Gump', 49, 2, 49, 'Vanessa Roth', None),
 ('Forrest Gump', 55, 2, 55, 'Michael Conner Humphreys', None),
 ('Forrest Gump', 60, 2, 60, 'Robb Skyler', None),
 ('Forrest Gump', 63, 2, 63, 'Byron Minns', None),
 ('Forrest Gump', 64, 2, 64, 'Troy Christian', None),
 ('Forrest Gump', 66, 2, 66, 'Timothy McNeil', None),
 ('Forrest Gump', 67, 2, 67, 'Don Fischer', None),
 ('Forrest Gump', 69, 2, 69, 'Bill Roberson', None),
 ('Fo

4. Vypiš všechny filmy, kde hraje Tom Hanks.

In [38]:
list(engine.execute('''
        SELECT a.name, m.title
        FROM movie_to_actor AS ma
        LEFT JOIN actors AS a ON a.id = ma.actor_id
        LEFT JOIN movies AS m ON m.id = ma.movie_id
        WHERE name == 'Tom Hanks'
        '''))

[('Tom Hanks', 'Forrest Gump'),
 ('Tom Hanks', 'Zelená míle'),
 ('Tom Hanks', 'Zachraňte vojína Ryana'),
 ('Tom Hanks', 'Chyť mě, když to dokážeš'),
 ('Tom Hanks', 'Toy Story: Příběh hraček'),
 ('Tom Hanks', 'Philadelphia'),
 ('Tom Hanks', 'Toy Story 2: Příběh hraček')]

5. Zkus shrnout hereckou kariéru Toma Hankse. Zjisti:

Počet filmů, ve kterých hrál
Rok prvního a posledního filmu, ve kterém hrál
Průměrné hodnocení filmů, ve kterých hrál

In [41]:
list(engine.execute('''
        SELECT a.name, COUNT(m.title), MIN(m.year), MAX(m.year), AVG(m.rating)
        FROM movie_to_actor AS ma
        LEFT JOIN actors AS a ON a.id = ma.actor_id
        LEFT JOIN movies AS m ON m.id = ma.movie_id
        WHERE name == 'Tom Hanks'
        GROUP BY name
        '''))

[('Tom Hanks', 7, 1993, 2002, 88.34285714285714)]

6. Načti tabulku countries z úkolů k EDA2 do lokální sqlite databáze a přepiš do SQL úkol číslo 3, tzn.:

Zjisti, v jakých zemích, které NEpatří do low_income group, mají průměrně méně než 3000 kalorií na den.
Tip: Lokální databázi vytvoříš pomocí metody to_sql, kde si zvolíš libovolné jméno tabulky a cestu k novému souboru pomocí sqlite:///tvé_jméno_souboru.db.

In [43]:
countries = pd.read_csv("countries.csv")
countries

,name,iso,world_6region,world_4region,income_groups,is_eu,is_oecd,eu_accession,year,area,...,alcohol_adults,bmi_men,bmi_women,car_deaths_per_100000_people,calories_per_day,infant_mortality,life_expectancy,life_expectancy_female,life_expectancy_male,un_accession
0,Afghanistan,AFG,south_asia,asia,low_income,False,False,NaN,2018,652860.0,...,0.03,20.62,21.07,NaN,2090.0,66.3,58.69,65.812,63.101,1946-11-19
1,Albania,ALB,europe_central_asia,europe,upper_middle_income,False,False,NaN,2018,28750.0,...,7.29,26.45,25.66,5.978,3193.0,12.5,78.01,80.737,76.693,1955-12-14
2,Algeria,DZA,middle_east_north_africa,africa,upper_middle_income,False,False,NaN,2018,2381740.0,...,0.69,24.60,26.37,NaN,3296.0,21.9,77.86,77.784,75.279,1962-10-08
3,Andorra,AND,europe_central_asia,europe,high_income,False,False,NaN,2017,470.0,...,10.17,27.63,26.43,NaN,NaN,2.1,82.55,NaN,NaN,1993-07-28
4,Angola,AGO,sub_saharan_africa,africa,upper_middle_income,False,False,NaN,2018,1246700.0,...,5.57,22.25,23.48,NaN,2473.0,96.0,65.19,64.939,59.213,1976-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Venezuela,VEN,america,americas,upper_middle_income,False,False,NaN,2018,912050.0,...,7.60,27.45,28.13,7.332,2631.0,12.9,75.91,79.079,70.950,1945-11-15
189,Vietnam,VNM,east_asia_pacific,asia,lower_middle_income,False,False,NaN,2018,330967.0,...,3.91,20.92,21.07,NaN,2745.0,17.3,74.88,81.203,72.003,1977-09-20
190,Yemen,YEM,middle_east_north_africa,asia,lower_middle_income,False,False,NaN,2018,527970.0,...,0.20,24.44,26.11,NaN,2223.0,33.8,67.14,66.871,63.875,1947-09-30
191,Zambia,ZMB,sub_saharan_africa,africa,lower_middle_income,False,False,NaN,2018,752610.0,...,3.56,20.68,23.05,11.260,1930.0,43.3,59.45,65.362,59.845,1964-12-01


In [56]:
engine = sqlalchemy.create_engine('sqlite:///countries_sql.sqlite')
countries.to_sql('countries_sql', con=engine, if_exists='replace')

In [58]:
list(engine.execute('''
        SELECT name, income_groups, calories_per_day 
        FROM countries_sql
        WHERE income_groups IS NOT 'low_income' AND calories_per_day < 3000        
        '''))

[('Angola', 'upper_middle_income', 2473.0),
 ('Antigua and Barbuda', 'high_income', 2417.0),
 ('Armenia', 'lower_middle_income', 2928.0),
 ('Bahamas', 'high_income', 2670.0),
 ('Barbados', 'high_income', 2937.0),
 ('Belize', 'upper_middle_income', 2751.0),
 ('Bolivia', 'lower_middle_income', 2256.0),
 ('Botswana', 'upper_middle_income', 2326.0),
 ('Brunei', 'high_income', 2985.0),
 ('Bulgaria', 'upper_middle_income', 2829.0),
 ('Cameroon', 'lower_middle_income', 2671.0),
 ('Cape Verde', 'lower_middle_income', 2609.0),
 ('Chile', 'high_income', 2979.0),
 ('Colombia', 'upper_middle_income', 2804.0),
 ('Congo', 'lower_middle_income', 2208.0),
 ('Costa Rica', 'upper_middle_income', 2848.0),
 ("Cote d'Ivoire", 'lower_middle_income', 2799.0),
 ('Cyprus', 'high_income', 2649.0),
 ('Djibouti', 'lower_middle_income', 2607.0),
 ('Dominica', 'upper_middle_income', 2931.0),
 ('Dominican Republic', 'upper_middle_income', 2614.0),
 ('Ecuador', 'upper_middle_income', 2344.0),
 ('El Salvador', 'lower_